Hey there, in this notebook, I tried to visualize the Crimes Against Women using the available datasets.

**In this notebook you will be able to see some insight with regards to crimes that happened against women between the year 2001 and 2014. For instance,**
1. **A graph with the total number of crimes VS Years**
2. **Top 20 Districts of India with the highest number of the case reported for Crimes Against Women.**
3. **Top 5 States with Highest Number of Rapes.**
4. **Top 5 States with Highest number of Dowry Death.**
5. **A 3D Scatter plot for top 20 Districts of India with Rape, Literacy, and Population, etc.**


**In case of any feedback, suggestion, and issue please write comments in GitHub or drop a mail to satyamprasadtiwari@gmail.com**


--- 
---




**Important Points**(please read it):

Data is gathered from three different sources.
 1. Crimes against women data from Data.gov.in(CSV File)
 2. India 2011 Census Data from kaggle.com(CSV File)
 3. Literacy Data(based on Census 2011) from census2011.co.in/district.php( Data was scraped and then stored into CSV file)

As data is gathered from different sources there could be some issue, I tried my best to minimize them. For example, there was spelling difference with multiple West Bengal districts and Cyberabad is not mentioned in Census data.

Another important thing is to be noted that, the Literacy and Population data is from the 2011 India Census and is used against all years between 2001 to 2014.


**Bonus Point**: This notebook is made using Plotly library, you can zoom-in the graph, hover the point using the mouse to see detailed information in the cartesian plane and last 3D plot can be rotated 360 using mouse and hovering over the Scatter point will show you multiple information with that particular point.






---



---


---



---



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data=data.iloc[:,1:]
tc=c11.groupby("State name").sum().reset_index().sort_values("State name")
c11p=c11[["District name","Population", "State name"]]
c11p.columns=["DISTRICT","Population", "State"]

In [ ]:
lit.columns=['Unnamed: 0','DISTRICT', 'State', 'Literacy']
lit.drop("Unnamed: 0", axis=1,inplace=True)

In [ ]:
lit["DISTRICT"]=lit["DISTRICT"].str.lstrip()
lit.drop("State",axis=1,inplace=True)

In [ ]:
tc.drop("District code", axis=1,inplace=True)

for i in range (0, len(c11["State name"])):
  c11["State name"][i]=c11["State name"][i].title()
  c11["District name"][i]=c11["District name"][i].title()


In [ ]:
for i in range (0, len(data['Year'])):
  data['DISTRICT'][i]=data['DISTRICT'][i].title()
  data['STATE/UT'][i]=data['STATE/UT'][i].title()

for i in range(0,len(data["Year"])):
  if data['DISTRICT'][i] in ["Total District(S)", "Zz Total", "Delhi Ut Total"]:
    data['DISTRICT'][i]="Total"

for i in range(0,len(data["Year"])):
  if data['STATE/UT'][i] =="D&N Haveli":
    data['STATE/UT'][i]="D & N Haveli"    

for i in range(0,len(data["Year"])):
  if data['STATE/UT'][i] =="Delhi Ut":
    data['STATE/UT'][i]="Delhi" 

for i in range(0,len(data["Year"])):
  if data['STATE/UT'][i] =="A&N Islands":
    data['STATE/UT'][i]="A & N Islands"        

In [ ]:
#this is done because there are a lot of UTs divided by direction, ie., Delhi ans Sikkim both has North, South district.

for i in range(0,len(data["Year"])):
  if data['DISTRICT'][i] in ["South", "North", "East", "West"]:
    data['DISTRICT'][i]= str(data["STATE/UT"][i]) + " "+  str(data['DISTRICT'][i])

In [ ]:
sd=data[["STATE/UT", "DISTRICT"]].drop_duplicates()

**Now we will add the literacy and population columns with main data**

Here few values are missing. This is because we are combining data from different sources and those are prepared by different authorities and that is why there is some difference in the Spelling of District and State/UT names.

For example,
1. Coochbehar- Koch Bihar
2. Malda- Maldah
3. 24 Parganas North-North Twenty Four Parganas
4. 24 Parganas South-South Twenty Four Parganas

The only way is to deal with this is manual entries!



In [ ]:
for i in range (0, len(data["DISTRICT"])):
  if data["DISTRICT"][i] == "Coochbehar":
    data["DISTRICT"][i]= "Koch Bihar"
    
  elif data["DISTRICT"][i] == "Malda":
    data["DISTRICT"][i]= "Maldah"
    
  elif data["DISTRICT"][i] in  ["24 Parganas North","North 24 Parganas"]:
    data["DISTRICT"][i]= "North Twenty Four Parganas"  
    
  elif data["DISTRICT"][i] in  ["24 Parganas South","South 24 Parganas"]:
    data["DISTRICT"][i]= "South Twenty Four Parganas"   
  elif data["DISTRICT"][i] == "Hooghly":
    data["DISTRICT"][i] = "Hugli"
    
  elif data["DISTRICT"][i] == "Hyderabad City":
    data["DISTRICT"][i] = "Hyderabad"

  elif data["DISTRICT"][i] == "Cyberabad":
    data["STATE/UT"][i] = "Andhra Pradesh"


In [ ]:
District_Total= data.groupby("DISTRICT").sum()

#Lets add Population and Literacy to every individual District

District_Total=pd.merge(District_Total,c11p, on = "DISTRICT", how="left")
District_Total=pd.merge(District_Total,lit, on = "DISTRICT", how="left")
# District_Total = pd.merge(District_Total,sd, on = "DISTRICT", how="left")
District_Total20=District_Total.sort_values(by="Rape", ascending=False).reset_index().head(22)
District_Total20.drop(0,axis=0,inplace=True)


In [ ]:
# We will use join(inner join) to concate two tables

state_total=data[data["DISTRICT"]=="Total"]
label10=np.arange(1,11)
label20=np.arange(0,20)


A lot of Literacy and Population values are missing, we will work on them manually if needed for analysis

**Lets see the line graph for years between 2001 to 2014**

In [ ]:
ydata=data.groupby("Year").sum().reset_index()

In [ ]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=ydata.Year, y=ydata.Rape,
                    mode='lines+markers',
                    name='Rape',
                    hovertext="Rape", ))
fig.add_trace(go.Scatter(x=ydata.Year, y=ydata["Dowry Deaths"],
                    mode='lines+markers',
                    name='Dowry Deaths'))
fig.add_trace(go.Scatter(x=ydata.Year, y=ydata["Kidnapping and Abduction"],
                    mode='lines+markers',
                     name='Kidnapping and Abduction',
                     hovertext="Kidnapping and Abduction"))
fig.add_trace(go.Scatter(x=ydata.Year, y=ydata["Importation of Girls"],
                    mode='lines+markers', 
                    name='Importation of Girls',
                    hovertext='Importation of Girls'))
fig.add_trace(go.Scatter(x=ydata.Year, y=ydata["Cruelty by Husband or his Relatives"],
                    mode='lines+markers', 
                    name='Cruelty by Husband or his Relatives',
                    hovertext='Cruelty by Husband or his Relatives'))
fig.add_trace(go.Scatter(x=ydata.Year, y=ydata["Insult to modesty of Women"],
                    mode='lines+markers',
                     name='Insult to modesty of Women',
                     hovertext='Insult to modesty of Women'))
fig.add_trace(go.Scatter(x=ydata.Year, y=ydata["Assault on women with intent to outrage her modesty"],
                    mode='lines+markers',
                     name='Assault on women '
                     , hovertext="Assault on women with intent to outrage her modesty"))


fig.update_layout(
    title="Line graph for years between 2001 to 2014 for all the Crimes against Women", 
    plot_bgcolor='rgba(0,0,0,0)')
fig.show()


#### We can see a sudden surge in Rape, Assault on women with intent to outrage her modesty and Kidnapping and Abduction after 2012. One of the main reasons for this could be **[Nirbhaya Delhi Case](https://en.wikipedia.org/wiki/2012_Delhi_gang_rape)**, after this, India got a moment of awareness and people started coming out against the crimes.

**top 20 district with total number of crimes happened against Womens**

In [ ]:
District_Total["Total"]=District_Total["Rape"]+ District_Total["Kidnapping and Abduction"] + District_Total["Dowry Deaths"]+District_Total["Insult to modesty of Women"]+District_Total["Cruelty by Husband or his Relatives"]+District_Total["Importation of Girls"]
srt=District_Total.sort_values(by="Total",ascending=False)
srt=srt.head(23).reset_index()
srt.drop([0,6,7],axis=0,inplace=True)
# srt.drop(7,axis=0,inplace=True)

In [ ]:
import plotly.express as px
fig=px.bar(srt, x="DISTRICT",y="Total" ,
           color='State', text='Total', 
           title="Top 20 district in India with total number of crimes happened against Womens",)
fig.update_traces( textposition='outside')
fig.update_layout(plot_bgcolor='white')

fig.show()

**Top 10 States/UT with Highest number of Rapes**




In [ ]:
state_total_r=state_total.groupby("STATE/UT").sum().sort_values(by="Rape", ascending=False).head(10).reset_index()

In [ ]:
fig=go.Figure(data=[go.Bar(x=state_total_r['STATE/UT'],y=state_total_r["Rape"], marker={'color':state_total_r['Rape']})])
fig.update_layout(title="Total number of Rape vs State/UT",xaxis_title="Name of State/UT", yaxis_title="Number of Rapes",
                  plot_bgcolor='white' )
fig.data[0].marker.line.width = 3
fig.data[0].marker.line.color = "black"

fig.show()

In [ ]:
        state_total_for_pie=state_total.groupby("STATE/UT").sum()
        state_total_for_pie["Total"]= state_total_for_pie["Rape"]+ state_total_for_pie["Kidnapping and Abduction"] + state_total_for_pie["Dowry Deaths"]+state_total_for_pie["Insult to modesty of Women"]+state_total_for_pie["Cruelty by Husband or his Relatives"]+state_total_for_pie["Importation of Girls"]
        state_total_for_pie=state_total_for_pie.sort_values(by="Total")
        fig=go.Figure(data=[go.Bar(x=state_total_for_pie.index,y=state_total_for_pie['Total'], marker={"color":np.arange(0, len(state_total_for_pie['Total']))})])
        fig.update_layout(autosize=True,plot_bgcolor='rgba(0,0,0,0)' )
        fig.show()

In [ ]:
        state_total_for_pie=state_total.groupby("STATE/UT").sum()
        state_total_for_pie["Total"]= state_total_for_pie["Rape"]+ state_total_for_pie["Kidnapping and Abduction"] + state_total_for_pie["Dowry Deaths"]+state_total_for_pie["Insult to modesty of Women"]+state_total_for_pie["Cruelty by Husband or his Relatives"]+state_total_for_pie["Importation of Girls"]
        state_total_for_pie=state_total_for_pie.sort_values(by="Total")
        fig=go.Figure(data=[go.Pie(labels=state_total_for_pie.index,values=state_total_for_pie['Total'],)])
        fig.update_traces(hoverinfo='label+percent', textinfo='label+percent', textfont_size=10, )
        fig.update_layout(autosize=True, )
        fig.show()

**Top 10 States/UT with Highest number of "Cruelty by Husband or his Relatives" cases**

In [ ]:
total_data_c=state_total.groupby("STATE/UT").sum().sort_values(by="Cruelty by Husband or his Relatives", ascending=False).head(10)

In [ ]:
fig=go.Figure(data=[go.Bar(x=total_data_c.index,y=total_data_c["Cruelty by Husband or his Relatives"] ,marker={'color':label10}	)])
fig.update_layout(title="Cruelty by Husband or his Relatives vs State/UT",
                  xaxis_title="Name of State/UT", yaxis_title="Cruelty by Husband or his Relatives" ,
                  plot_bgcolor='white' )
fig.data[0].marker.line.width = 3
fig.data[0].marker.line.color = "black"
fig.show()

**Top 10 states with Highest number of Dowry Deaths**




In [ ]:
state_total_d=state_total.groupby("STATE/UT").sum().sort_values(by="Dowry Deaths", ascending=False).head(10)

In [ ]:
fig=go.Figure(data=[go.Bar(x=state_total_d.index,y=state_total_d["Dowry Deaths"], marker={'color':label10}	)])
fig.update_layout(title="Dowry Deaths vs State/UT",xaxis_title="Name of State/UT", yaxis_title="Dowry Deaths" ,
                                    plot_bgcolor='white' )
fig.data[0].marker.line.width = 3
fig.data[0].marker.line.color = "black"
fig.show()

**Top 10 states with Highest number of Kidnapping and Abduction**



In [ ]:
total_data_k=state_total.groupby("STATE/UT").sum().sort_values(by="Kidnapping and Abduction", ascending=False).head(10)

In [ ]:
fig=go.Figure(data=[go.Bar(x=total_data_k.index,y=total_data_k["Kidnapping and Abduction"] ,marker={'color':label10}	)])
fig.update_layout(title="Kidnapping and Abduction vs State/UT",xaxis_title="Name of State/UT", yaxis_title="Kidnapping and Abduction" ,
                                    plot_bgcolor='white' )
fig.data[0].marker.line.width = 3
fig.data[0].marker.line.color = "black"
fig.show()

# **Madhya Pradesh had the highest number of rapes between 2001 to 2014, lets find out the top 20 district that were involved in this!**

In [ ]:
MP_data=data[data['STATE/UT']=="Madhya Pradesh"]
MP_data_District=MP_data.groupby("DISTRICT").sum().sort_values(by="Rape", ascending=False).head(20)
MP_data_District=pd.merge(MP_data_District,lit,on="DISTRICT", how="inner") # Adding Literacy of Madhya Pradesh's Districts
MP_data_District=pd.merge(MP_data_District,c11p,on="DISTRICT", how="inner") # Adding Population of Madhya Pradesh's  Districts

In [ ]:
fig=go.Figure(data=[go.Scatter(x=MP_data_District.DISTRICT,y=MP_data_District.Rape,
                               
                               mode='markers',
                               marker=dict(size=(MP_data_District.Rape/25), color=label20 ))])
fig.update_layout(title="Top 20 Districts with Highest number of Rapes in Madhya Pradesh",xaxis_title="Name of Districts", yaxis_title="Number of Rapes" ,
                   plot_bgcolor='white' )
fig.data[0].marker.line.width = 3
fig.data[0].marker.line.color = "black"
fig.show()

Betul has the highest number of rape case reported from Madhya Pradesh!

# **Now We gonna find Districts (with State, Population, Literacy) those have the highest number of rapes reported in Entire India**

In [ ]:
District_Total= data.groupby("DISTRICT").sum().sort_values(by="Rape", ascending=False).reset_index().head(21)
District_Total.drop(0,axis=0,inplace=True)

In [ ]:
#Lets add Population and Literacy to every individual District

District_Total=pd.merge(District_Total,c11p, on = "DISTRICT", how="left")
District_Total=pd.merge(District_Total,lit, on = "DISTRICT", how="left")
District_Total.head(3)

,DISTRICT,Year,Rape,Kidnapping and Abduction,Dowry Deaths,Assault on women with intent to outrage her modesty,Insult to modesty of Women,Cruelty by Husband or his Relatives,Importation of Girls,Population,State,Literacy
0,Murshidabad,28105,3819,3317,455,4417,56,21607,0,7103807.0,WEST BENGAL,66.59
1,South Twenty Four Parganas,28105,2550,3121,742,1955,35,24154,3,8161961.0,WEST BENGAL,77.51
2,Mumbai Commr.,18090,2384,1435,149,5989,1778,3901,1,NaN,NaN,NaN




Note- 'Mumbai Commr.'	is not a district so we will drop it for now.



In [ ]:
District_Total.drop(2, inplace=True)
District_Total.drop("Year", axis=1, inplace=True)

In [ ]:
import plotly.express as px


fig= px.bar(District_Total,x="DISTRICT",y=District_Total["Rape"],
                                color=District_Total["Rape"],
                                hover_data=["State","Population","Literacy"], 
               )
fig.update_layout(title="Top 20 Districts(State, Population, Literacy) with Highest number of Rapes in India",
                  xaxis_title="Name of Districts",
                  yaxis_title="Number of Rapes" ,
                   plot_bgcolor='white'
                   )
fig.data[0].marker.line.width = 3
fig.data[0].marker.line.color = "black"
fig.show()

# **Ploting a line graph for India's top Rape Reported district to see if there any relation of Rape with Literacy**. 
It has two lines: Rape and Literacy of District

In [ ]:
temp_District_Total=District_Total

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
District_TotalMM = pd.DataFrame(scaler.fit_transform(District_Total[["Literacy","Population","Rape"]]), columns=["Literacy","Population","Rape"])
District_TotalMM.set_index(District_Total.index, inplace = True)

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp_District_Total.DISTRICT, y=District_TotalMM.Rape,
                    mode='lines+markers',
                    name='Rape',
                    hovertext="Rape"))

fig.add_trace(go.Scatter(x=temp_District_Total.DISTRICT, y=District_TotalMM.Literacy,
                    mode='lines+markers',
                     name='Literacy'))

# Edit the layout
fig.update_layout(title='Line Plot between Rape and Literacy(India)',
                   xaxis_title='Districts Name',
                   yaxis_title='Normalized values in between 0 and 1',
                  plot_bgcolor='white'
                   )


fig.show()

Last graph shows that there is no relation of Rape withh Literacy, we will try once more with Madhya Pradesh's data to see if there is any pattern.

In [ ]:
temp_MP_data_District=MP_data_District

MP_data_DistrictMM = pd.DataFrame(scaler.fit_transform(MP_data_District[["Literacy","Population","Rape"]]), columns=["Literacy","Population","Rape"])
MP_data_DistrictMM.set_index(MP_data_District.index, inplace = True)


# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp_MP_data_District.DISTRICT, y=MP_data_DistrictMM.Rape,
                    mode='lines',
                    name='Rape',
                    hovertext="Rape"))

fig.add_trace(go.Scatter(x=temp_MP_data_District.DISTRICT, y=MP_data_DistrictMM.Literacy,
                    mode='lines',
                     name='Literacy'))

# Edit the layout
fig.update_layout(title='Line Plot between Rapen and Literacy(Madhya Pradesh)',
                   xaxis_title='Districts Name',
                   yaxis_title='Normalized values in between 0 and 1',
                  plot_bgcolor='white'
                   )


fig.show()

**Again, there is nothing common in Rape and Literacy !**.

---




## **Ploting a line graph for India's top Rape Reported district to see if there any relation of Rape with Literacy.**

It has two lines: Rape and Population of District

In [ ]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp_District_Total.DISTRICT, y=District_TotalMM.Rape,
                    mode='lines',
                    name='Rape',
                    hovertext="Rape"))

fig.add_trace(go.Scatter(x=temp_District_Total.DISTRICT, y=District_TotalMM.Population,
                    mode='lines',
                     name='Population'))

# Edit the layout
fig.update_layout(title='Line Plot between Rape and Population(India)',
                   xaxis_title='Districts Name',
                   yaxis_title='Normalized values in between 0 and 1', plot_bgcolor='white',)


fig.show()

In [ ]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp_MP_data_District.DISTRICT, y=MP_data_DistrictMM.Rape,
                    mode='lines',
                    name='Rape',
                    hovertext="Rape"))

fig.add_trace(go.Scatter(x=temp_MP_data_District.DISTRICT, y=MP_data_DistrictMM.Population,
                    mode='lines',
                     name='Population'))

# Edit the layout
fig.update_layout(title='Line Plot between Rapen and Population(Madhya Pradesh)',
                   xaxis_title='Districts Name',
                   yaxis_title='Normalized values in between 0 and 1', plot_bgcolor='white',)


fig.show()

## 3D Scatter Plot for top 20 Rape Reported District!

In [ ]:
fig = px.scatter_3d(District_Total, x='Rape', y='Literacy', z='Population',
              color='DISTRICT', symbol="State", )
fig.update_layout(plot_bgcolor='white', paper_bgcolor='rgba(0,0,0,0)', height=700)
fig.show()

---

---



# **End of  Notebook**

---

---